参考： https://www.jianshu.com/p/9e72e1adbc8f 得知数据集在keras中已经包含了，可以直接使用

## 读取数据

In [1]:
from keras.datasets import boston_housing

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test)=boston_housing.load_data()

In [19]:
x_train.shape,x_test.shape

((404, 13), (102, 13))

In [18]:
x_test[:2]

array([[1.80846e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.79000e-01,
        6.43400e+00, 1.00000e+02, 1.83470e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 2.72500e+01, 2.90500e+01],
       [1.23290e-01, 0.00000e+00, 1.00100e+01, 0.00000e+00, 5.47000e-01,
        5.91300e+00, 9.29000e+01, 2.35340e+00, 6.00000e+00, 4.32000e+02,
        1.78000e+01, 3.94950e+02, 1.62100e+01]])

In [5]:
y_train[:2]

array([15.2, 42.3])

可以看到，x有13个特征，都是float，y也是数字，这是个回归模型，关注的是住房价格（单位是千美元）,也不用对Y使用LabelENcoder变成数字再用to_category变成独热码了

In [6]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.cross_validation import KFold,cross_val_score #warning CV模块移除，改为model_selection 但是修改过的函数使用方法变了
# from sklearn.model_selection import KFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

D:\anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


因为导入的数据是直接从keras中读取的，所以只使用x_train和y_train就好了

## 基准模型

In [14]:
def baseline_model():
    model=Sequential()
    model.add(Dense(13,input_dim=13,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,kernel_initializer='normal'))
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model
# 输出层没有激活函数，因为在回归问题中我们希望直接取结果
# 编译使用的 损失函数是 MSE 与要优化的函数一致，可以对模型的预测有直观的理解，因为MSE乘方就是千美元的误差

In [15]:
seed=7
np.random.seed(7)
model=KerasRegressor(build_fn=baseline_model,epochs=150,batch_size=5,verbose=0)

In [16]:
kfold=KFold(n=len(x_train),n_folds=10,random_state=seed,shuffle=True) 
results=cross_val_score(model,x_test,y_train,cv=kfold)

In [17]:
print('mean:%.2f std:%.2f'%(results.mean(),results.std()))

mean:-17.38 std:5.07


+ 第一次运行： mean:-17.41 std:5.19 出现负值。。这就比较尴尬了
+ 第二次运行：mean:-17.38 std:5.07 还是负值，可能因为数据太少了。。。没办法

## 增加数据预处理

In [21]:
pipeline=Pipeline([('standardize',StandardScaler()),
                  ('mlp',KerasRegressor(build_fn=baseline_model,epochs=150,batch_size=5,verbose=0))
                  ])
kfold=KFold(n=len(x_train),n_folds=10,random_state=seed,shuffle=True) 
results=cross_val_score(pipeline,x_train,y_train,cv=kflod)

In [22]:
print('mean:%.2f std:%.2f'%(results.mean(),results.std()))

mean:-12.77 std:5.26


看起来MSE均值变小了，数据标准化还是很有用的。。。

## 调整模型拓扑

神经网络有很多可调的参数：最有弹性的就是网络结构。可以在深度和宽度上进行调整（就是层数和每层的神经元个数）

### deeper

增加网络层数可以提高效果，这样模型可以提取并组合更多的特征

In [24]:
def baseline_model():
    model=Sequential()
    model.add(Dense(13,input_dim=13,kernel_initializer='normal',activation='relu'))
    model.add(Dense(6,kernel_initializer='normal',activation='relu')) #加一个隐层试试
    model.add(Dense(1,kernel_initializer='normal'))
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model

pipeline=Pipeline([('standardize',StandardScaler()),
                  ('mlp',KerasRegressor(build_fn=baseline_model,epochs=150,batch_size=5,verbose=0))
                  ])
kfold=KFold(n=len(x_train),n_folds=10,random_state=seed,shuffle=True) 
results=cross_val_score(pipeline,x_train,y_train,cv=kflod)

In [25]:
print('mean:%.2f std:%.2f'%(results.mean(),results.std()))

mean:-11.02 std:3.97


### wider

In [26]:
def baseline_model():
    model=Sequential()
    model.add(Dense(20,input_dim=13,kernel_initializer='normal',activation='relu')) #把这层变宽
    model.add(Dense(1,kernel_initializer='normal'))
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model

pipeline=Pipeline([('standardize',StandardScaler()),
                  ('mlp',KerasRegressor(build_fn=baseline_model,epochs=150,batch_size=5,verbose=0))
                  ])
kfold=KFold(n=len(x_train),n_folds=10,random_state=seed,shuffle=True) 
results=cross_val_score(pipeline,x_train,y_train,cv=kflod)

In [27]:
print('mean:%.2f std:%.2f'%(results.mean(),results.std()))

mean:-11.15 std:4.42


在我的代码里，更深比更宽效果好。